In [ ]:
!pip install -q transformers
# !pip install -q pytorch-lightning

# Code for TPU packages install
# !curl -q https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
# !python pytorch-xla-env-setup.py --apt-packages libomp5 libopenblas-dev

In [ ]:
# Checking out the GPU we have access to
!nvidia-smi

Sun May 31 11:07:55 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.82       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P8    69W / 149W |      0MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [ ]:
# Importing stock libraries
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler

from transformers import T5Tokenizer, T5ForConditionalGeneration

# Preparing for TPU usage
# import torch_xla
# import torch_xla.core.xla_model as xm
# device = xm.xla_device()

In [ ]:
# # Setting up the device for GPU usage

from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [ ]:
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/news_summary.csv',encoding='latin-1')
# df.head()

In [ ]:
df = df[['text','ctext']]
df.ctext = 'summarize: ' + df.ctext
df.head()

,text,ctext
0,The Administration of Union Territory Daman an...,summarize: The Daman and Diu administration on...
1,Malaika Arora slammed an Instagram user who tr...,summarize: From her special numbers to TV?appe...
2,The Indira Gandhi Institute of Medical Science...,summarize: The Indira Gandhi Institute of Medi...
3,Lashkar-e-Taiba's Kashmir commander Abu Dujana...,summarize: Lashkar-e-Taiba's Kashmir commander...
4,Hotels in Maharashtra will train their staff t...,summarize: Hotels in Mumbai and other Indian c...


In [ ]:
# Sections of config

# Defining some key variables that will be used later on in the training

tokenizer = T5Tokenizer.from_pretrained("t5-base")
MAX_LEN = 512
SUMMARY_LEN = 150
TRAIN_BATCH_SIZE = 2
VALID_BATCH_SIZE = 2
EPOCHS = 2
LEARNING_RATE = 1e-4

In [ ]:
# Creating a custom dataset for reading the dataframe and loading it into the dataloader to pass it to the neural network at a later stage for finetuning the model and to prepare it for predictions


class CustomDataset(Dataset):

    def __init__(self, dataframe, tokenizer, source_len, summ_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.source_len = source_len
        self.summ_len = summ_len
        self.text = self.data.text
        self.ctext = self.data.ctext

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        ctext = str(self.ctext[index])
        ctext = ' '.join(ctext.split())

        text = str(self.text[index])
        text = ' '.join(text.split())

        source = self.tokenizer.batch_encode_plus([ctext], max_length= self.source_len, pad_to_max_length=True,return_tensors='pt')
        target = self.tokenizer.batch_encode_plus([text], max_length= self.summ_len, pad_to_max_length=True,return_tensors='pt')

        source_ids = source['input_ids'].squeeze()
        source_mask = source['attention_mask'].squeeze()
        target_ids = target['input_ids'].squeeze()
        target_mask = target['attention_mask'].squeeze()

        return {
            'source_ids': source_ids.to(dtype=torch.long), 
            'source_mask': source_mask.to(dtype=torch.long), 
            'target_ids': target_ids.to(dtype=torch.long),
            'target_ids_y': target_ids.to(dtype=torch.long)
        }

In [ ]:
# Creating the dataset and dataloader for the neural network

train_size = 0.8
train_dataset=df.sample(frac=train_size,random_state=42).reset_index(drop=True)
test_dataset=df.drop(train_dataset.index).reset_index(drop=True)


print("FULL Dataset: {}".format(df.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(test_dataset.shape))

training_set = CustomDataset(train_dataset, tokenizer, MAX_LEN, SUMMARY_LEN)
testing_set = CustomDataset(test_dataset, tokenizer, MAX_LEN, SUMMARY_LEN)

FULL Dataset: (4514, 2)
TRAIN Dataset: (3611, 2)
TEST Dataset: (903, 2)


In [ ]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': False,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

In [ ]:
model = T5ForConditionalGeneration.from_pretrained("t5-base")
model = model.to(device)

In [ ]:
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [ ]:
def train(epoch):
    model.train()
    for _,data in enumerate(training_loader, 0):
        y = data['target_ids'].to(device, dtype = torch.long)
        y_ids = y[:, :-1].contiguous()
        lm_labels = y[:, 1:].clone().detach()
        lm_labels[y[:, 1:] == tokenizer.pad_token_id] = -100
        ids = data['source_ids'].to(device, dtype = torch.long)
        mask = data['source_mask'].to(device, dtype = torch.long)

        outputs = model(input_ids = ids, attention_mask = mask, decoder_input_ids=y_ids, lm_labels=lm_labels)
        loss = outputs[0]
        
        if _%500==0:
            print(f'Epoch: {epoch}, Loss:  {loss.item()}')
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # xm.optimizer_step(optimizer)
        # xm.mark_step()

In [ ]:
for epoch in range(EPOCHS):
    train(epoch)

Epoch: 0, Loss:  7.654313564300537
Epoch: 0, Loss:  1.0062143802642822
Epoch: 0, Loss:  1.8140990734100342
Epoch: 0, Loss:  1.5853947401046753
Epoch: 1, Loss:  1.0318087339401245
Epoch: 1, Loss:  1.0657906532287598
Epoch: 1, Loss:  1.2831671237945557
Epoch: 1, Loss:  1.473008632659912


In [ ]:
def writer(predictions, actuals):
    # output_path = r'/content/drive/My Drive/Colab Notebooks/'
    # pred_file = output_path+'preds.txt'
    # actual_file = output_path+'actuals.txt'
    # with open(pred_file, 'w') as f:
    #     for pred in predictions:
    #         f.write('%s\n' % pred)

    # with open(actual_file, 'w') as f:
    #     for actual in actuals:
    #         f.write('%s\n' % actual)
    my_dict = {
        'Actual Headline': actuals,
        'Prediction': predictions       
        }
    final_output = pd.DataFrame(my_dict)

    return final_output


In [ ]:
def validate(epoch):
    model.eval()
    predictions = []
    actuals = []
    with torch.no_grad():
        for _, data in enumerate(testing_loader, 0):
            y = data['target_ids'].to(device, dtype = torch.long)
            ids = data['source_ids'].to(device, dtype = torch.long)
            mask = data['source_mask'].to(device, dtype = torch.long)

            generated_ids = model.generate(
                input_ids = ids,
                attention_mask = mask, 
                max_length=150, 
                num_beams=2,
                repetition_penalty=2.5, 
                length_penalty=1.0, 
                early_stopping=True
                )
            preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]
            target = [tokenizer.decode(t, skip_special_tokens=True, clean_up_tokenization_spaces=True)for t in y]
            if _%100==0:
                print(f'Completed {_}')

            predictions.extend(preds)
            actuals.extend(target)
    return predictions, actuals

In [ ]:
for epoch in range(1):
    predictions, actuals = validate(epoch)
    final_df = writer(predictions, actuals)
    final_df.to_csv('/content/drive/My Drive/Colab Notebooks/predictions.csv')
    print('Output Files generated for review')

Completed 0
Completed 100
Completed 200
Completed 300
Completed 400
Output Files generated for review


In [ ]:
# from transformers import T5Tokenizer, T5ForConditionalGeneration

# tokenizer = T5Tokenizer.from_pretrained('t5-small')
# model = T5ForConditionalGeneration.from_pretrained('t5-small')
# input_ids = tokenizer.encode("Hello, my dog is cute", return_tensors="pt")  # Batch size 1
# outputs = model(input_ids=input_ids, decoder_input_ids=input_ids, lm_labels=input_ids)
# loss, prediction_scores = outputs[:2]

# tokenizer = T5Tokenizer.from_pretrained('t5-small')
# model = T5ForConditionalGeneration.from_pretrained('t5-small')
# input_ids = tokenizer.encode_plus("summarize: Hello, my dog is cute", return_tensors="pt")  # Batch size 1
# outputs = model.generate(input_ids['input_ids'])
# pred = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in outputs]

In [ ]:
# print(f'The loss is: {loss}')
# print(f'The Prediction score is {prediction_scores}')
# print(f'The output is {outputs}')
# print(f'The final output is {pred}')